In [1]:
# in this script, we will calculate the feedbacks as temperature contributions

#by: Ty Janoski
# updated: 07.03.20

In [2]:
# import statements
import xarray as xr
import scipy.stats as stats
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = "pdf"

In [3]:
models = ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR', 'GFDL-ESM2G',
         'MIROC-ESM', 'FGOALS-g2','bcc-csm1-1','BNU-ESM','CanESM2','CCSM4',
          'CSIRO-Mk3-6-0','FGOALS-s2','GFDL-CM3','inmcm4',
         'IPSL-CM5A-LR','MIROC5','MPI-ESM-LR','MPI-ESM-P','MRI-CGCM3','NorESM1-M']
arc_norm_list = []
glb_norm_list = []
arc_ml_list = []
glb_ml_list = []
for m in models:
    # read in spatially averaged feedbacks, as computed in spatial_avg_feedbacks.ipynb
    arc = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_arc_feedbacks.nc',
                         use_cftime=True,drop_variables=['ml_conv'])
    glb = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_glb_feedbacks.nc',
                         use_cftime=True,drop_variables=['ml_conv'])
    # to convert to temperature contributions, we normalize by the global Planck feedbacks
    # divided by the global change in surface skin temp
    norm = (glb.planck_LWAS / glb.dTs)
    
    arc_norm = -1 * arc / norm
    glb_norm = -1 * glb / norm
    
    # a bit of wonkiness: planck is actually the deviation from the global planck feedback
    arc_norm['planck_LWAS'] = -1 * ((arc.planck_LWAS / arc.dTs) - (glb.planck_LWAS / glb.dTs)) * arc.dTs / norm
    glb_norm['planck_LWAS'] = -1 * ((glb.planck_LWAS / glb.dTs) - (glb.planck_LWAS / glb.dTs)) * glb.dTs / norm
    
    # it would be useful to combine each model into one dataset with a "model" dimension
    # each model's calendar is different, so unfortunately we will override the time dimension
    arc_norm['time'] = np.arange(0,1800,1)
    glb_norm['time'] = np.arange(0,1800,1)
    
    # append to lists
    arc_norm_list.append(arc_norm)
    glb_norm_list.append(glb_norm)
    
    ot_models = ['ACCESS1-0','ACCESS1-3','GFDL-CM3','MIROC5',
             'MPI-ESM-LR','MPI-ESM-P','MIROC-ESM','BNU-ESM',
             'CCSM4','MRI-CGCM3','FGOALS-g2',
             'CNRM-CM5','NorESM1-M','CSIRO-Mk3-6-0']
    if(m in ot_models):
        print(m)
        # read in spatially averaged feedbacks, as computed in spatial_avg_feedbacks.ipynb
        arc_ml = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_arc_feedbacks.nc',
                             use_cftime=True).ml_conv
        glb_ml = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_glb_feedbacks.nc',
                             use_cftime=True).ml_conv
        arc_ml_norm = -1 * arc_ml/norm
        glb_ml_norm = -1 * glb_ml/norm
        arc_ml_norm['time'] = np.arange(0,1800,1)
        glb_ml_norm['time'] = np.arange(0,1800,1)
        arc_ml_list.append(arc_ml_norm)
        glb_ml_list.append(glb_ml_norm)
# combine along new model coordinate
arc_temp = xr.concat(arc_norm_list,dim='model')
arc_temp['model'] = models
glb_temp = xr.concat(glb_norm_list,dim='model')
glb_temp['model'] = models
arc_ml_temp = xr.concat(arc_ml_list,dim='model')
arc_ml_temp['model'] = ot_models
glb_ml_temp = xr.concat(glb_ml_list,dim='model')
glb_ml_temp['model'] = ot_models


ACCESS1-0
ACCESS1-3
CNRM-CM5
MIROC-ESM
FGOALS-g2
BNU-ESM
CCSM4
CSIRO-Mk3-6-0
GFDL-CM3
MIROC5
MPI-ESM-LR
MPI-ESM-P
MRI-CGCM3
NorESM1-M


In [7]:
diff = arc_temp - glb_temp
diff_ml = arc_ml_temp - glb_ml_temp
# tvals = diff.mean(dim='model') / (diff.std(dim='model') / np.sqrt(len(diff.model)))
# tvals_ml = diff_ml.mean(dim='model') / (diff_ml.std(dim='model') / np.sqrt(len(diff_ml.model)))
tvals = diff.isel(time=slice(0,3)).mean(dim=['model','time']) / (diff.isel(time=slice(0,3)).mean(dim='time').std(dim='model') / np.sqrt(len(diff.model)))
tvals_ml = diff_ml.isel(time=slice(0,3)).mean(dim=['model','time']) / (diff_ml.isel(time=slice(0,3)).mean(dim='time').std(dim='model') / np.sqrt(len(diff_ml.model)))

In [13]:
diff.lapse_LWAS.isel(time=slice(0,3)).mean(dim=['time']).std(dim='model')

<xarray.DataArray 'lapse_LWAS' ()>
array(0.51310316)

In [9]:
# critical t-value for 95% confidence intervals = 2.086, n=21, df=20
# for n=14,df=13, tcrit = 2.160
for var in tvals.variables:
    print(var,float(tvals[var]))
print('ocean_conv',float(tvals_ml))

dTs 3.9522666676162808
wv_LWAS 0.037116858262261535
wv_LWCS -0.04445264156899255
wv_SWAS 0.7590012378674438
wv_SWCS 0.9727520407946434
wv_NETAS 0.19018927553330442
wv_NETCS 0.10982171305823939
planck_LWAS 0.7039238991345066
lapse_LWAS 4.501711710517133
albedo_SWAS -2.026587629711138
albedo_SWCS -1.8592300281660266
cloud_LW 2.133745144722932
cloud_SW -6.329875379554538
cloud_NET -4.003616699950214
atmos_conv -0.04682863437751996
ocean_conv 1.3140082123313968


In [7]:
fig, axes = plt.subplots(ncols=3,nrows=1,figsize=(7.4,3))

for ax in axes.flatten():
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.xaxis.set_tick_params(bottom=True, top=False)
    ax.yaxis.set_tick_params(left=True, right=False)
    ax.spines['bottom'].set_position('zero') # spine for xaxis 
    ax.spines['left'].set_position('zero')  # spine for yaxis 
    ax.tick_params(axis='both',labelsize=8)
    ax.grid(linestyle=(0,(1,5)))


axes[2].set_title(r'(c) Last 30 years',fontsize=8)

last30arc = arc_temp.isel(time=slice(1200,1800)).mean(dim=['time','model'])
last30arcml = arc_ml_temp.isel(time=slice(1200,1800)).mean(dim=['time','model'])
last30glb = glb_temp.isel(time=slice(1200,1800)).mean(dim=['time','model'])
last30glbml = glb_ml_temp.isel(time=slice(1200,1800)).mean(dim=['time','model'])

axes[2].scatter(last30glb.planck_LWAS,
                  last30arc.planck_LWAS,label='P\'',color='darkorange',s=75)
axes[2].scatter(last30glb.lapse_LWAS,
                  last30arc.lapse_LWAS,color='green',label='LR',s=75)
axes[2].scatter(last30glb.albedo_SWAS,
                  last30arc.albedo_SWAS,color='red',label='Albedo',s=75)
axes[2].scatter(last30glb.wv_NETAS,
                  last30arc.wv_NETAS,color='purple',label='WV',s=75)
axes[2].scatter(last30glb.cloud_NET,
                  last30arc.cloud_NET,color='turquoise', label='Cloud',s=75)
axes[2].scatter(last30glb.atmos_conv,
                  last30arc.atmos_conv,color='orchid',label='AHT',s=75)
axes[2].scatter(last30glbml,
                  last30arcml,color='Navy',label='OHT',s=75)
axes[2].legend(loc='upper left',fontsize=7)
axes[2].scatter(last30glbml,
                  last30arcml,color='Navy',label='OHT',facecolors='white',linewidth=1.5,s=75)

axes[2].set_xlim([-8,8])
axes[2].set_ylim([-8,8])
axes[2].set_xticks(np.arange(-8,9,2))
axes[2].plot(np.arange(-8,10.5,0.5),np.arange(-8,10.5,0.5),color='black',linestyle='--')
# #############################################################################

first1arc = arc_temp.isel(time=slice(0,12)).mean(dim=['time','model'])
first1arcml = arc_ml_temp.isel(time=slice(0,12)).mean(dim=['time','model'])
first1glb = glb_temp.isel(time=slice(0,12)).mean(dim=['time','model'])
first1glbml = glb_ml_temp.isel(time=slice(0,12)).mean(dim=['time','model'])

axes[1].set_title('(b) First year',fontsize=8)

axes[1].scatter(first1glb.planck_LWAS,
                  first1arc.planck_LWAS,label='P\'',s=75,color='darkorange')
axes[1].scatter(first1glb.lapse_LWAS,
                  first1arc.lapse_LWAS,color='green',label='LR',s=75)
axes[1].scatter(first1glb.albedo_SWAS,
                  first1arc.albedo_SWAS,color='red',label='Albedo',s=75)
axes[1].scatter(first1glb.wv_NETAS,
                  first1arc.wv_NETAS,color='purple',label='WV',s=75,facecolors='none',linewidth=1.5)
axes[1].scatter(first1glb.cloud_NET,
                  first1arc.cloud_NET,color='turquoise', label='Cloud',s=75)
axes[1].scatter(first1glb.atmos_conv,
                  first1arc.atmos_conv,color='orchid',label='AHT',facecolors='none',s=75,linewidth=1.5)
axes[1].scatter(first1glbml,
                  first1arcml,color='Navy',label='OHT',s=75)
axes[1].plot(np.arange(-4.5,10.5,0.5),np.arange(-4.5,10.5,0.5),color='black',linestyle='--')
axes[1].set_xlim([-1.5,1.5])
axes[1].set_ylim([-1.5,1.5])

# axes[2].legend(loc='upper left',fontsize=7)

# # ######################################################################################################################

first3arc = arc_temp.isel(time=slice(0,3)).mean(dim=['time','model'])
first3arcml = arc_ml_temp.isel(time=slice(0,3)).mean(dim=['time','model'])
first3glb = glb_temp.isel(time=slice(0,3)).mean(dim=['time','model'])
first3glbml = glb_ml_temp.isel(time=slice(0,3)).mean(dim=['time','model'])

axes[0].set_title('(a) First 3 months',fontsize=8)

axes[0].scatter(first3glb.planck_LWAS,
                  first3arc.planck_LWAS,label='P\'',s=75,color='darkorange',facecolors='none',linewidth=1.5)
axes[0].scatter(first3glb.lapse_LWAS,
                  first3arc.lapse_LWAS,color='green',label='LR',s=75)
axes[0].scatter(first3glb.albedo_SWAS,
                  first3arc.albedo_SWAS,color='red',label='Albedo',s=75,facecolors='none',linewidth=1.5)
axes[0].scatter(first3glb.wv_NETAS,
                  first3arc.wv_NETAS,color='purple',label='WV',s=75,facecolors='none',linewidth=1.5)
axes[0].scatter(first3glb.cloud_NET,
                  first3arc.cloud_NET,color='turquoise', label='Cloud',s=75)
axes[0].scatter(first3glb.atmos_conv,
                  first3arc.atmos_conv,color='orchid',label='AHT',facecolors='none',s=75,linewidth=1.5)
axes[0].scatter(first3glbml,
                  first3arcml,color='Navy',label='OHT',facecolors='none',linewidth=1.5,s=75)
axes[0].plot(np.arange(-4.5,10.5,0.5),np.arange(-4.5,10.5,0.5),color='black',linestyle='--')
axes[0].set_xticks(np.arange(-0.9,1,0.3))
axes[0].set_yticks(np.arange(-0.9,1,0.3))
axes[0].set_xlim([-0.9,0.9])
axes[0].set_ylim([-0.9,0.9])

# ######################################################################################################################
# # Let's see if we can add text labels to this
axes[0].text(-0.35,0.45,'Arctic (K)',fontsize=8, rotation=90)
axes[1].text(-0.583,0.75,'Arctic (K)',fontsize=8, rotation=90)
axes[2].text(-2,4,'Arctic (K)',fontsize=8, rotation=90)

axes[0].text(0.3,-0.23,'Global (K)',fontsize=8)
axes[1].text(0.5,-0.383,'Global (K)',fontsize=8)
axes[2].text(3,-1.9,'Global (K)', fontsize=8)

axes[0].text(0.6,0.7,'+AA',fontsize=7,rotation=46)
axes[0].text(0.72,0.62,'-AA',fontsize=7,rotation=46)
axes[1].text(1.0,1.167,'+AA',fontsize=7,rotation=46)
axes[1].text(1.2,1.033,'-AA',fontsize=7,rotation=46)
axes[2].text(5.33,6.22,'+AA',fontsize=7,rotation=46)
axes[2].text(6.4,5.511,'-AA',fontsize=7,rotation=46)

plt.tight_layout()
plt.show()
# plt.savefig('Fig_5_070320.svg')

<Figure size 532.8x216 with 3 Axes>

In [3]:
models = ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR', 'GFDL-ESM2G',
         'MIROC-ESM', 'FGOALS-g2','bcc-csm1-1','BNU-ESM','CanESM2','CCSM4',
          'CSIRO-Mk3-6-0','FGOALS-s2','GFDL-CM3','inmcm4',
         'IPSL-CM5A-LR','MIROC5','MPI-ESM-LR','MPI-ESM-P','MRI-CGCM3','NorESM1-M']
arc_norm_list = []
glb_norm_list = []
arc_ml_list = []
glb_ml_list = []
for m in models:
    # read in spatially averaged feedbacks, as computed in spatial_avg_feedbacks.ipynb
    arc = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_shell_arc_feedbacks.nc',
                         use_cftime=True,drop_variables=['ml_conv'])
    glb = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_shell_glb_feedbacks.nc',
                         use_cftime=True,drop_variables=['ml_conv'])
    # to convert to temperature contributions, we normalize by the global Planck feedbacks
    # divided by the global change in surface skin temp
    norm = (glb.planck_LWAS / glb.dTs)
    
    arc_norm = -1 * arc / norm
    glb_norm = -1 * glb / norm
    
    # a bit of wonkiness: planck is actually the deviation from the global planck feedback
    arc_norm['planck_LWAS'] = -1 * ((arc.planck_LWAS / arc.dTs) - (glb.planck_LWAS / glb.dTs)) * arc.dTs / norm
    glb_norm['planck_LWAS'] = -1 * ((glb.planck_LWAS / glb.dTs) - (glb.planck_LWAS / glb.dTs)) * glb.dTs / norm
    
    # it would be useful to combine each model into one dataset with a "model" dimension
    # each model's calendar is different, so unfortunately we will override the time dimension
    arc_norm['time'] = np.arange(0,1800,1)
    glb_norm['time'] = np.arange(0,1800,1)
    
    # append to lists
    arc_norm_list.append(arc_norm)
    glb_norm_list.append(glb_norm)
    
    ot_models = ['ACCESS1-0','ACCESS1-3','GFDL-CM3','MIROC5',
             'MPI-ESM-LR','MPI-ESM-P','MIROC-ESM','BNU-ESM',
             'CCSM4','MRI-CGCM3','FGOALS-g2',
             'CNRM-CM5','NorESM1-M','CSIRO-Mk3-6-0']
    if(m in ot_models):
        print(m)
        # read in spatially averaged feedbacks, as computed in spatial_avg_feedbacks.ipynb
        arc_ml = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_arc_feedbacks.nc',
                             use_cftime=True).ml_conv
        glb_ml = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_glb_feedbacks.nc',
                             use_cftime=True).ml_conv
        arc_ml_norm = -1 * arc_ml/norm
        glb_ml_norm = -1 * glb_ml/norm
        arc_ml_norm['time'] = np.arange(0,1800,1)
        glb_ml_norm['time'] = np.arange(0,1800,1)
        arc_ml_list.append(arc_ml_norm)
        glb_ml_list.append(glb_ml_norm)
# combine along new model coordinate
arc_temp = xr.concat(arc_norm_list,dim='model')
arc_temp['model'] = models
glb_temp = xr.concat(glb_norm_list,dim='model')
glb_temp['model'] = models
arc_ml_temp = xr.concat(arc_ml_list,dim='model')
arc_ml_temp['model'] = ot_models
glb_ml_temp = xr.concat(glb_ml_list,dim='model')
glb_ml_temp['model'] = ot_models

ACCESS1-0
ACCESS1-3
CNRM-CM5
MIROC-ESM
FGOALS-g2
BNU-ESM
CCSM4
CSIRO-Mk3-6-0
GFDL-CM3
MIROC5
MPI-ESM-LR
MPI-ESM-P
MRI-CGCM3
NorESM1-M


In [14]:
fig, ax = plt.subplots(figsize=(3,3))

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.xaxis.set_tick_params(bottom=True, top=False)
ax.yaxis.set_tick_params(left=True, right=False)
ax.spines['bottom'].set_position('zero') # spine for xaxis 
ax.spines['left'].set_position('zero')  # spine for yaxis 
ax.tick_params(axis='both',labelsize=8)
ax.grid()

last30arc = arc_temp.isel(time=slice(1200,1800)).mean(dim=['time','model'])
last30arcml = arc_ml_temp.isel(time=slice(1200,1800)).mean(dim=['time','model'])
last30glb = glb_temp.isel(time=slice(1200,1800)).mean(dim=['time','model'])
last30glbml = glb_ml_temp.isel(time=slice(1200,1800)).mean(dim=['time','model'])

ax.scatter(last30glb.planck_LWAS,
                  last30arc.planck_LWAS,label='P\'',color='darkorange',s=75)
ax.scatter(last30glb.lapse_LWAS,
                  last30arc.lapse_LWAS,color='green',label='LR',s=75)
ax.scatter(last30glb.albedo_SWAS,
                  last30arc.albedo_SWAS,color='red',label='Albedo',s=75)
ax.scatter(last30glb.wv_NETAS,
                  last30arc.wv_NETAS,color='purple',label='WV',s=75)
ax.scatter(last30glb.cloud_NET,
                  last30arc.cloud_NET,color='turquoise', label='Cloud',s=75)
ax.scatter(last30glb.atmos_conv,
                  last30arc.atmos_conv,color='orchid',label='AHT',s=75)
ax.scatter(last30glbml,
                  last30arcml,color='Navy',label='OHT',facecolors='none',linewidth=1.5,s=75)

ax.set_xlim([-8,8])
ax.set_ylim([-8,8])
ax.set_xticks(np.arange(-8,9,2))
ax.plot(np.arange(-8,10.5,0.5),np.arange(-8,10.5,0.5),color='black',linestyle='--')

# ######################################################################################################################
# # Let's see if we can add text labels to this
ax.text(-2,4,'Arctic (K)',fontsize=8, rotation=90)

ax.text(3,-1.9,'Global (K)', fontsize=8)
ax.set_title('Last 30 years',fontsize=8)

ax.text(5.33,6.22,'+AA',fontsize=7,rotation=46)
ax.text(6.4,5.511,'-AA',fontsize=7,rotation=46)

plt.tight_layout()
plt.savefig('feedback_scatter_CAM3_albedo_060820.pdf')

<Figure size 216x216 with 1 Axes>

In [ ]:
models = ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR', 'GFDL-ESM2G',
         'MIROC-ESM', 'FGOALS-g2','bcc-csm1-1','BNU-ESM','CanESM2','CCSM4',
          'CSIRO-Mk3-6-0','FGOALS-s2','GFDL-CM3','inmcm4',
         'IPSL-CM5A-LR','MIROC5','MPI-ESM-LR','MPI-ESM-P','MRI-CGCM3','NorESM1-M']
arc_norm_list = []
glb_norm_list = []
arc_ml_list = []
glb_ml_list = []
for m in models:
    # read in spatially averaged feedbacks, as computed in spatial_avg_feedbacks.ipynb
    arc = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_arc_feedbacks.nc',
                         use_cftime=True,drop_variables=['ml_conv'])
    glb = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_glb_feedbacks.nc',
                         use_cftime=True,drop_variables=['ml_conv'])
    # to convert to temperature contributions, we normalize by the global Planck feedbacks
    # divided by the global change in surface skin temp
    norm = (glb.planck_LWAS / glb.dTs)
    
    arc_norm = -1 * arc / norm
    glb_norm = -1 * glb / norm
    
    # a bit of wonkiness: planck is actually the deviation from the global planck feedback
    arc_norm['planck_LWAS'] = -1 * ((arc.planck_LWAS / arc.dTs) - (glb.planck_LWAS / glb.dTs)) * arc.dTs / norm
    glb_norm['planck_LWAS'] = -1 * ((glb.planck_LWAS / glb.dTs) - (glb.planck_LWAS / glb.dTs)) * glb.dTs / norm
    
    # it would be useful to combine each model into one dataset with a "model" dimension
    # each model's calendar is different, so unfortunately we will override the time dimension
    arc_norm['time'] = np.arange(0,1800,1)
    glb_norm['time'] = np.arange(0,1800,1)
    
    # append to lists
    arc_norm_list.append(arc_norm)
    glb_norm_list.append(glb_norm)
    
    ot_models = ['ACCESS1-0','ACCESS1-3','GFDL-CM3','MIROC5',
             'MPI-ESM-LR','MPI-ESM-P','MIROC-ESM','BNU-ESM',
             'CCSM4','MRI-CGCM3','FGOALS-g2',
             'CNRM-CM5','NorESM1-M','CSIRO-Mk3-6-0']
    if(m in ot_models):
        print(m)
        # read in spatially averaged feedbacks, as computed in spatial_avg_feedbacks.ipynb
        arc_ml = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_arc_feedbacks.nc',
                             use_cftime=True).ml_conv
        glb_ml = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_glb_feedbacks.nc',
                             use_cftime=True).ml_conv
        arc_ml_norm = -1 * arc_ml/norm
        glb_ml_norm = -1 * glb_ml/norm
        arc_ml_norm['time'] = np.arange(0,1800,1)
        glb_ml_norm['time'] = np.arange(0,1800,1)
        arc_ml_list.append(arc_ml_norm)
        glb_ml_list.append(glb_ml_norm)
# combine along new model coordinate
arc_temp = xr.concat(arc_norm_list,dim='model')
arc_temp['model'] = models
glb_temp = xr.concat(glb_norm_list,dim='model')
glb_temp['model'] = models
arc_ml_temp = xr.concat(arc_ml_list,dim='model')
arc_ml_temp['model'] = ot_models
glb_ml_temp = xr.concat(glb_ml_list,dim='model')
glb_ml_temp['model'] = ot_models


In [ ]:
models = ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR', 'GFDL-ESM2G',
         'MIROC-ESM', 'FGOALS-g2','bcc-csm1-1','BNU-ESM','CanESM2','CCSM4',
          'CSIRO-Mk3-6-0','FGOALS-s2','GFDL-CM3','inmcm4',
         'IPSL-CM5A-LR','MIROC5','MPI-ESM-LR','MPI-ESM-P','MRI-CGCM3','NorESM1-M']
subarc_norm_list = []
trop_norm_list = []
subarc_ml_list = []
trop_ml_list = []
for m in models:
    # read in spatially averaged feedbacks, as computed in spatial_avg_feedbacks.ipynb
    subarc = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_subarc_feedbacks.nc',
                         use_cftime=True,drop_variables=['ml_conv'])
    trop = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_trop_feedbacks.nc',
                         use_cftime=True,drop_variables=['ml_conv'])
    glb = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_glb_feedbacks.nc',
                           use_cftime=True,drop_variables=['ml_conv'])
    # to convert to temperature contributions, we normalize by the global Planck feedbacks
    # divided by the global change in surface skin temp
    norm = (glb.planck_LWAS / glb.dTs)
    
    subarc_norm = -1 * subarc / norm
    trop_norm = -1 * trop / norm
    
    # a bit of wonkiness: planck is actually the deviation from the global planck feedback
    subarc_norm['planck_LWAS'] = -1 * ((subarc.planck_LWAS / subarc.dTs) - (glb.planck_LWAS / glb.dTs)) * subarc.dTs / norm
    trop_norm['planck_LWAS'] = -1 * ((trop.planck_LWAS / trop.dTs) - (glb.planck_LWAS / glb.dTs)) * trop.dTs / norm
    
    # it would be useful to combine each model into one dataset with a "model" dimension
    # each model's calendar is different, so unfortunately we will override the time dimension
    subarc_norm['time'] = np.arange(0,1800,1)
    trop_norm['time'] = np.arange(0,1800,1)
    
    # append to lists
    subarc_norm_list.append(subarc_norm)
    trop_norm_list.append(trop_norm)
    
    ot_models = ['ACCESS1-0','ACCESS1-3','GFDL-CM3','MIROC5',
             'MPI-ESM-LR','MPI-ESM-P','MIROC-ESM','BNU-ESM',
             'CCSM4','MRI-CGCM3','FGOALS-g2',
             'CNRM-CM5','NorESM1-M','CSIRO-Mk3-6-0']
    if(m in ot_models):
        print(m)
        # read in spatially averaged feedbacks, as computed in spatial_avg_feedbacks.ipynb
        subarc_ml = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_subarc_feedbacks.nc',
                             use_cftime=True).ml_conv
        trop_ml = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_trop_feedbacks.nc',
                             use_cftime=True).ml_conv
        subarc_ml_norm = -1 * subarc_ml/norm
        trop_ml_norm = -1 * trop_ml/norm
        subarc_ml_norm['time'] = np.arange(0,1800,1)
        trop_ml_norm['time'] = np.arange(0,1800,1)
        subarc_ml_list.append(subarc_ml_norm)
        trop_ml_list.append(trop_ml_norm)
# combine along new model coordinate
subarc_temp = xr.concat(subarc_norm_list,dim='model')
subarc_temp['model'] = models
trop_temp = xr.concat(trop_norm_list,dim='model')
trop_temp['model'] = models
subarc_ml_temp = xr.concat(subarc_ml_list,dim='model')
subarc_ml_temp['model'] = ot_models
trop_ml_temp = xr.concat(trop_ml_list,dim='model')
trop_ml_temp['model'] = ot_models

In [ ]:
fig,ax = plt.subplots(figsize=(4,4))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.xaxis.set_tick_params(bottom=True, top=False)
ax.yaxis.set_tick_params(left=True, right=False)
ax.spines['bottom'].set_position('zero') # spine for xaxis 
ax.spines['left'].set_position('zero')  # spine for yaxis 
ax.tick_params(axis='both',labelsize=8)
ax.grid()

last30arc = subarc_temp.isel(time=slice(1200,1799)).mean(dim=['time','model'])
last30arcml = subarc_ml_temp.isel(time=slice(1200,1799)).mean(dim=['time','model'])
last30glb = trop_temp.isel(time=slice(1200,1799)).mean(dim=['time','model'])
last30glbml = trop_ml_temp.isel(time=slice(1200,1799)).mean(dim=['time','model'])

ax.set_title('P&M Plot',fontsize=10)

ax.scatter(last30glb.planck_LWAS,
                  last30arc.planck_LWAS,label='P\'',color='darkorange',s=75)
ax.scatter(last30glb.lapse_LWAS,
                  last30arc.lapse_LWAS,color='green',label='LR',s=75)
ax.scatter(last30glb.albedo_SWAS,
                  last30arc.albedo_SWAS,color='red',label='Albedo',s=75)
ax.scatter(last30glb.wv_NETAS,
                  last30arc.wv_NETAS,color='purple',label='WV',s=75)
ax.scatter(last30glb.cloud_NET,
                  last30arc.cloud_NET,color='turquoise', label='Cloud',s=75)
ax.scatter(last30glb.atmos_conv,
                  last30arc.atmos_conv,color='orchid',label='AHT',s=75,linewidth=1.5)
ax.scatter(last30glbml,
                  last30arcml,color='Navy',label='OHT',linewidth=1.5,s=75)
ax.set_xlim([-4,8])
ax.set_ylim([-4,8])
ax.text(2,-1.3,'Tropical Warming (K)',fontsize=9)
ax.text(-1.3,3.5,'Arctic Warming (K)',fontsize=9,rotation=90)
ax.plot(np.arange(-4.5,10.5,0.5),np.arange(-4.5,10.5,0.5),color='black',linestyle='--')


plt.tight_layout()
plt.savefig('/home/tylerj/CMIP5_AA_final/pithan_mauritsen_plot_060320.pdf')